In [1]:
import pandas as pd
from functools import reduce
import datetime
from datetime import timedelta
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
# from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv("Statistik_Harian_per_Provinsi_COVID19_Indonesia_Rev.csv")

In [3]:
df.columns

Index(['Object_ID', 'Provinsi', 'Tanggal', 'Kasus_Terkonfirmasi_Akumulatif',
       'Penambahan_Harian_Kasus_Terkonf', 'Kasus_Sembuh_Akumulatif',
       'Penambahan_Harian_Kasus_Sembuh', 'Kasus_Meninggal_Akumulatif',
       'Penambahan_Harian_Kasus_Meningg', 'Kasus_Aktif_Akumulatif',
       'CFR_Harian', 'RI_Harian', 'FID', 'ObjectId'],
      dtype='object')

In [4]:
df = df[['Provinsi', 'Tanggal', 'Kasus_Terkonfirmasi_Akumulatif',
       'Penambahan_Harian_Kasus_Terkonf', 'Kasus_Sembuh_Akumulatif',
       'Penambahan_Harian_Kasus_Sembuh', 'Kasus_Meninggal_Akumulatif',
       'Penambahan_Harian_Kasus_Meningg', 'Kasus_Aktif_Akumulatif']]

In [5]:
df.Tanggal = pd.to_datetime(df.Tanggal).dt.date

In [6]:
df = df.drop_duplicates()

In [7]:
df = df.dropna()

In [8]:
prov = df.Provinsi.unique()

In [9]:
data = []

for p in prov:
    dt = df.loc[df.Provinsi == p]
    dt = dt[["Tanggal",'Kasus_Terkonfirmasi_Akumulatif']]
    dt.columns = ['Tanggal',p]
#     print(dt)
    data.append(dt)

In [10]:
df_all =  reduce(lambda left,right : pd.merge(left,right, on=["Tanggal"],how='outer'),data).fillna(0)

In [11]:
df_make = df_all[7:]

In [12]:
# dft = df_all.loc[(df_all.Tanggal < datetime.date(2020, 3, 9) ) & (df_all.Tanggal >= datetime.date(2020, 3, 9) - timedelta(days=7))]

In [13]:
data_train = []
for index,row in df_make.iterrows():
    dft = df_all.loc[(df_all.Tanggal < row['Tanggal'] ) & (df_all.Tanggal >= row['Tanggal'] - timedelta(days=7))].sum()
    dft.drop("Indonesia",inplace=True)
    inp = dft.to_dict()
    inp["Tanggal"] = row['Tanggal']
    data_train.append(inp)

In [14]:
df_1week = pd.DataFrame(data_train)

In [15]:
df_train = pd.merge(df[['Provinsi','Tanggal','Kasus_Terkonfirmasi_Akumulatif']],df_1week,how='inner',on="Tanggal")

In [16]:
df_train.head()

,Provinsi,Tanggal,Kasus_Terkonfirmasi_Akumulatif,Aceh,Bali,Banten,Bengkulu,DKI Jakarta,Daerah Istimewa Yogyakarta,Gorontalo,...,Papua Barat,Riau,Sulawesi Barat,Sulawesi Selatan,Sulawesi Tengah,Sulawesi Tenggara,Sulawesi Utara,Sumatera Barat,Sumatera Selatan,Sumatera Utara
0,Aceh,2020-03-09,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Bali,2020-03-09,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Banten,2020-03-09,1.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Kepulauan Bangka Belitung,2020-03-09,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bengkulu,2020-03-09,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
from sklearn.preprocessing import OneHotEncoder as SklearnOneHotEncoder

class OneHotEncoder(SklearnOneHotEncoder):
    def __init__(self, **kwargs):
        super(OneHotEncoder, self).__init__(**kwargs)
        self.fit_flag = False

    def fit(self, X, **kwargs):
        out = super().fit(X)
        self.fit_flag = True
        return out

    def transform(self, X, **kwargs):
        sparse_matrix = super(OneHotEncoder, self).transform(X)
        new_columns = self.get_new_columns(X=X)
        d_out = pd.DataFrame(sparse_matrix.toarray(), columns=new_columns, index=X.index)
        return d_out

    def fit_transform(self, X, **kwargs):
        self.fit(X)
        return self.transform(X)

    def get_new_columns(self, X):
        new_columns = []
        for i, column in enumerate(X.columns):
            j = 0
            while j < len(self.categories_[i]):
                new_columns.append(f'{column}_<{self.categories_[i][j]}>')
                j += 1
        return new_columns

In [18]:
encoder = OneHotEncoder()
dfe = encoder.fit_transform(df_train.Provinsi.to_frame())

In [19]:
df_train = pd.concat([df_train,dfe],axis=1)

In [20]:
df_train.drop("Provinsi",axis=1,inplace=True)

In [21]:
df_train['first_date'] = pd.to_datetime("3-2-2020").date()

In [22]:
df_train['days_after_first_case'] = df_train['Tanggal'] - df_train['first_date']

In [23]:
df_train['days_after_first_case'] = df_train['days_after_first_case'].dt.days

In [24]:
df_train

,Tanggal,Kasus_Terkonfirmasi_Akumulatif,Aceh,Bali,Banten,Bengkulu,DKI Jakarta,Daerah Istimewa Yogyakarta,Gorontalo,Jambi,...,Provinsi_<Sulawesi Barat>,Provinsi_<Sulawesi Selatan>,Provinsi_<Sulawesi Tengah>,Provinsi_<Sulawesi Tenggara>,Provinsi_<Sulawesi Utara>,Provinsi_<Sumatera Barat>,Provinsi_<Sumatera Selatan>,Provinsi_<Sumatera Utara>,first_date,days_after_first_case
0,2020-03-09,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-02,7
1,2020-03-09,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-02,7
2,2020-03-09,1.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-02,7
3,2020-03-09,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-02,7
4,2020-03-09,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-02,7
5,2020-03-09,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-02,7
6,2020-03-09,16.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-02,7
7,2020-03-09,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-02,7
8,2020-03-09,2.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-02,7
9,2020-03-09,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-02,7


In [25]:
df_train.columns

Index(['Tanggal', 'Kasus_Terkonfirmasi_Akumulatif', 'Aceh', 'Bali', 'Banten',
       'Bengkulu', 'DKI Jakarta', 'Daerah Istimewa Yogyakarta', 'Gorontalo',
       'Jambi', 'Jawa Barat', 'Jawa Tengah', 'Jawa Timur', 'Kalimantan Barat',
       'Kalimantan Selatan', 'Kalimantan Tengah', 'Kalimantan Timur',
       'Kalimantan Utara', 'Kepulauan Bangka Belitung', 'Kepulauan Riau',
       'Lampung', 'Maluku', 'Maluku Utara', 'Nusa Tenggara Barat',
       'Nusa Tenggara Timur', 'Papua', 'Papua Barat', 'Riau', 'Sulawesi Barat',
       'Sulawesi Selatan', 'Sulawesi Tengah', 'Sulawesi Tenggara',
       'Sulawesi Utara', 'Sumatera Barat', 'Sumatera Selatan',
       'Sumatera Utara', 'Provinsi_<Aceh>', 'Provinsi_<Bali>',
       'Provinsi_<Banten>', 'Provinsi_<Bengkulu>', 'Provinsi_<DKI Jakarta>',
       'Provinsi_<Daerah Istimewa Yogyakarta>', 'Provinsi_<Gorontalo>',
       'Provinsi_<Indonesia>', 'Provinsi_<Jambi>', 'Provinsi_<Jawa Barat>',
       'Provinsi_<Jawa Tengah>', 'Provinsi_<Jawa Timur>',

In [26]:
df_train.dtypes

Tanggal                                  object
Kasus_Terkonfirmasi_Akumulatif          float64
Aceh                                    float64
Bali                                    float64
Banten                                  float64
Bengkulu                                float64
DKI Jakarta                             float64
Daerah Istimewa Yogyakarta              float64
Gorontalo                               float64
Jambi                                   float64
Jawa Barat                              float64
Jawa Tengah                             float64
Jawa Timur                              float64
Kalimantan Barat                        float64
Kalimantan Selatan                      float64
Kalimantan Tengah                       float64
Kalimantan Timur                        float64
Kalimantan Utara                        float64
Kepulauan Bangka Belitung               float64
Kepulauan Riau                          float64
Lampung                                 

In [28]:
X = df_train[['Aceh', 'Bali', 'Banten',
       'Bengkulu', 'DKI Jakarta', 'Daerah Istimewa Yogyakarta', 'Gorontalo',
       'Jambi', 'Jawa Barat', 'Jawa Tengah', 'Jawa Timur', 'Kalimantan Barat',
       'Kalimantan Selatan', 'Kalimantan Tengah', 'Kalimantan Timur',
       'Kalimantan Utara', 'Kepulauan Bangka Belitung', 'Kepulauan Riau',
       'Lampung', 'Maluku', 'Maluku Utara', 'Nusa Tenggara Barat',
       'Nusa Tenggara Timur', 'Papua', 'Papua Barat', 'Riau', 'Sulawesi Barat',
       'Sulawesi Selatan', 'Sulawesi Tengah', 'Sulawesi Tenggara',
       'Sulawesi Utara', 'Sumatera Barat', 'Sumatera Selatan',
       'Sumatera Utara', 'Provinsi_<Aceh>', 'Provinsi_<Bali>',
       'Provinsi_<Banten>', 'Provinsi_<Bengkulu>', 'Provinsi_<DKI Jakarta>',
       'Provinsi_<Daerah Istimewa Yogyakarta>', 'Provinsi_<Gorontalo>',
       'Provinsi_<Indonesia>', 'Provinsi_<Jambi>', 'Provinsi_<Jawa Barat>',
       'Provinsi_<Jawa Tengah>', 'Provinsi_<Jawa Timur>',
       'Provinsi_<Kalimantan Barat>', 'Provinsi_<Kalimantan Selatan>',
       'Provinsi_<Kalimantan Tengah>', 'Provinsi_<Kalimantan Timur>',
       'Provinsi_<Kalimantan Utara>', 'Provinsi_<Kepulauan Bangka Belitung>',
       'Provinsi_<Kepulauan Riau>', 'Provinsi_<Lampung>', 'Provinsi_<Maluku>',
       'Provinsi_<Maluku Utara>', 'Provinsi_<Nusa Tenggara Barat>',
       'Provinsi_<Nusa Tenggara Timur>', 'Provinsi_<Papua>',
       'Provinsi_<Papua Barat>', 'Provinsi_<Riau>',
       'Provinsi_<Sulawesi Barat>', 'Provinsi_<Sulawesi Selatan>',
       'Provinsi_<Sulawesi Tengah>', 'Provinsi_<Sulawesi Tenggara>',
       'Provinsi_<Sulawesi Utara>', 'Provinsi_<Sumatera Barat>',
       'Provinsi_<Sumatera Selatan>', 'Provinsi_<Sumatera Utara>', 'days_after_first_case']]
# y = 1 * x_0 + 2 * x_1 + 3
y = df_train['Kasus_Terkonfirmasi_Akumulatif']
# reg = ElasticNet().fit(X,y)
reg = LinearRegression().fit(X, y)
reg.score(X, y)

0.5472447645781138

In [31]:
# pd.DataFrame(zip(X.columns,reg.coef_))

,0,1
0,Aceh,-0.197800
1,Bali,-0.313122
2,Banten,-0.332127
3,Bengkulu,0.410610
4,DKI Jakarta,-0.030062
5,Daerah Istimewa Yogyakarta,-0.334561
6,Gorontalo,0.207784
7,Jambi,-0.189546
8,Jawa Barat,0.019952
9,Jawa Tengah,0.047703
